# Playbooks


As we already learn in the other lessons, playbooks are yaml files where we group tasks.

A playbook starts with an hosts mark specifying the hosts to run the playbook to.

```
---
- hosts: web
  tasks: 
  ...
```


For yaml, a playbook is a *list* of `hosts` entries.


```
- hosts: localhost
  tasks:
  - name: one or more tasks to be run on localhost
    ...
    
- hosts: web
  tasks:
  - name: followed by tasks to be run on web hostgroup
    ...
```

We can even pick a single element in an host group, or add `tags` entries to restrict executions

```
- hosts: db[0]
  tags: beware
  tasks:
  - name: .. and then on the first db node ;)

```

In [2]:
cd exercise-07

/notebooks/exercise-07


### Goal

 - user and skel
 - install packages and enable service (not on containers)
 - lineinfile
 - curl
 - file, copy & fetch
 
Creating small reports.

More fun with:

  - iterations
  - ignore_errors
  - with_fileglob and remote_fileglob ;)

Creating small reports.



## Gathering facts


When run, a playbook gathers facts about hosts (remember the setup module?).

Gathering facts may be time-consuming, so you can [tune it via ansible.cfg](http://docs.ansible.com/ansible/intro_configuration.html#gathering) or disable it

```
- hosts: web
  gather_facts: no
  tasks:
  ...
```

We can use the [predefined variables](http://docs.ansible.com/ansible/playbooks_variables.html#magic-variables-and-how-to-access-information-about-other-hosts) too:

  - group_names
  - groups
  - hostvars
  - environment 

In [5]:
!ansible-playbook debug.yml --tags gather_facts


PLAY [those tasks are run on localhost only] ***********************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [This is the debug module.] ***********************************************
ok: [localhost] => {
    "msg": "A string"
}

TASK [The debug module can print a variable too] *******************************
ok: [localhost] => {
    "ansible_hostname": "sysadminpy"
}

PLAY [those tasks are run on every web host] ***********************************

TASK [Gathering Facts] *********************************************************
fatal: [ansible101_web_3]: UNREACHABLE! => {"changed": false, "msg": "Authentication or permission failure. In some cases, you may have been able to authenticate and did not have permissions on the remote directory. Consider changing the remote temp path in ansible.cfg to a path rooted in \"/tmp\". Failed command was: ( umask 77 && mkdir -p \"` echo ~/.ansible/tmp/ansible-tmp-14990062

## Exercise: 

[modify debug.yml](/edit/notebooks/exercise-06/debug.yml) to disable fact gathering and use the following cell to test it 

In [6]:
# Test the exercise here

### Hints

 - use ansible_facts as possible instead of gathering facts with uname & co
 - test and template your iterations statically instead of continuosly gathering facts
 

## hostvars

Ansible allows referencing facts between hosts, so that we're able to get eg. a list of all webserver ip addresses.

Between set variables we have:

  - hostvars
  - groups
  


In [3]:
!ansible-playbook debug.yml --tags hostvars


PLAY [those tasks are run on localhost only] ***********************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

PLAY [those tasks are run on every web host] ***********************************

TASK [Gathering Facts] *********************************************************
fatal: [ansible101_web_3]: UNREACHABLE! => {"changed": false, "msg": "Authentication or permission failure. In some cases, you may have been able to authenticate and did not have permissions on the remote directory. Consider changing the remote temp path in ansible.cfg to a path rooted in \"/tmp\". Failed command was: ( umask 77 && mkdir -p \"` echo ~/.ansible/tmp/ansible-tmp-1499004149.75-90392408564316 `\" && echo ansible-tmp-1499004149.75-90392408564316=\"` echo ~/.ansible/tmp/ansible-tmp-1499004149.75-90392408564316 `\" ), exited with result 1", "unreachable": true}
ok: [ansible101_web_2]
ok: [ansible101_web_1]

PLAY [Run only on first host] **

## Exercise

Did you note anything about the hostvars keys printed out by the previous command?

## Filters

We can process hostvars using special functions named filters. 

Filters are based on the jinja2 template engine.


```
- name: We already found a simple filter mapping entries to int
  debug: msg="{{ 3.1415 | int }}"
```

A filter is essentially a function returning a function, like a lambda.

```
int_filter = lambda x: int(x)
```

A more complex filter: 
```
- name: This is a getter
  debug: msg="{{ ['host1', 'host2'] | map('extract', hostvars, ['key1', .. , 'keyN']) }}"
```

where
```
hostvars_getter = lambda host: hostvars[host]['key1'][..]['keyN']
```

You can pipeline filters and test incrementally.

In [9]:
!ansible-playbook debug.yml --tags filters


PLAY [those tasks are run on localhost only] ***********************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

PLAY [those tasks are run on every web host] ***********************************

TASK [Gathering Facts] *********************************************************
fatal: [ansible101_web_3]: UNREACHABLE! => {"changed": false, "msg": "Authentication or permission failure. In some cases, you may have been able to authenticate and did not have permissions on the remote directory. Consider changing the remote temp path in ansible.cfg to a path rooted in \"/tmp\". Failed command was: ( umask 77 && mkdir -p \"` echo ~/.ansible/tmp/ansible-tmp-1499009135.14-257218177738666 `\" && echo ansible-tmp-1499009135.14-257218177738666=\"` echo ~/.ansible/tmp/ansible-tmp-1499009135.14-257218177738666 `\" ), exited with result 1", "unreachable": true}
ok: [ansible101_web_2]
ok: [ansible101_web_1]

PLAY [Run only on first host]

## Packages, User and Files

Creating user and installing packages is easy

```
  - name: Install apache
    apt: item="{{item}}" state=present
    with_items:
    - apache2
    - curl
    
  - name: Remove wget
    apt: item=wget state=absent
```

In [8]:
!cat package-user.yml


- hosts: web
  tasks:
  - name: Install apache and other packages eventually iterating
    apt: name="{{item}}"
    with_items:
    - apache2
    - curl

  - name: Remove wget
    apt: name=wget state=absent

  - name: Create user
    user: 
      name: foo
      groups: adm
      append: yes
      shell: /bin/bash
      skeleton: /root   # duplicate /root/ content for the foo user

  - name: >
      Copy a file to ~foo expandig tilde.
      Do not forget quotes!
    copy:
      src: package-user.yml
      dest: "{{ '~foo' | expanduser }}"

  - name: >
      We can fetch files too (eg. logs):
      - files are dispatched to different directories.
      - we can avoid enforcing checksum as we're messing with 
        containers.
      
    fetch:
      src: /etc/passwd
      dest: /tmp/fetched
      validate_checksum: no


In [9]:
!ansible-playbook package-user.yml



PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
ok: [ansible101_web_1]

TASK [Install apache and other packages eventually iterating] ******************
ok: [ansible101_web_1] => (item=[u'apache2', u'curl'])

TASK [Remove wget] *************************************************************
ok: [ansible101_web_1]

TASK [Create user] *************************************************************
ok: [ansible101_web_1]

TASK [Copy a file to ~foo expandig tilde. Do not forget quotes!] ***************
ok: [ansible101_web_1]

TASK [We can fetch files too (eg. logs): - files are dispatched to different directories. - we can avoid enforcing checksum as we're messing with 
  containers.] ***
changed: [ansible101_web_1]

PLAY RECAP *********************************************************************
ansible101_web_1           : ok=6    changed=1    unreachable=0    failed=0   



In [12]:
!tree /tmp/fetched/

/tmp/fetched/
└── ansible101_web_1
    ├── etc
    └── var
        └── log

4 directories, 0 files


## Files and Directories

Creating files and directories. 

lineinfile / blockinfile

with_fileglob

In [10]:
!cat files-content.yml

- hosts: localhost
  name: >
    A playbook can be made of multiple stanzas. They'll be
    pipelined. 
  tasks:
  - name: Delete previous template
    file: state=absent dest=/tmp/index.html.template

  - name: |
      We're now just showing how to create a file containing a block of lines
    blockinfile:
      dest: /tmp/index.html.template
      create: yes
      owner: root
      group: root
      marker: no
      mode: 0644
      block: >
       <html>
       <body>
       System installed by {{ansible_hostname}} on {{ '{{' }} ansible_hostname {{ '}}' }}
       </body>
       </html>

  - name: Register command output in a variable
    shell: cat index.html.template
    args:
      chdir: /tmp 
    register: o_cat

  - name: Show output
    debug: var=o_cat.stdout
     

- hosts: web
  tasks:
  - name: Install apache and other packages eventually iterating
    apt: name="{{item}}"
    with_items:
    - apache2
    - curl
  
  - name: Facts are per-host variables
    set_fact: doc

In [11]:
!ansible-playbook -i inventory files-content.yml 



PLAY [A playbook can be made of multiple stanzas. They'll be pipelined.] *******

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [Delete previous template] ************************************************
changed: [localhost]

TASK [We're now just showing how to create a file containing a block of lines] ***
changed: [localhost]

TASK [Register command output in a variable] ***********************************
changed: [localhost]

TASK [Show output] *************************************************************
ok: [localhost] => {
    "o_cat.stdout": "False\n<html> <body> System installed by sysadminpy on {{ ansible_hostname }} </body> </html>\nFalse"
}

PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
fatal: [ansible101_web_3]: UNREACHABLE! => {"changed": false, "msg": "Authentication or permission failure. In some c

## shell module reloaded

Ansible can be used to reproduce issues and gather command output. 

Though the standard workflow can be done registering output in a temporary variable


In [14]:
!cat shell-output-01.yml

# 
- hosts: localhost
  tasks:
  - name: Run multiple commands via /bin/sh (no bashisms)
    shell: |
      ls -latr /etc/host*
    ignore_errors: yes
    register: o
    
  - name: Show output in a single block...
    debug: var=o.stdout

  - name: ... or one per line
    debug: var=item
    with_items: >
      {{ o.stdout_lines }}



In [15]:
!ansible-playbook shell-output-01.yml


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [Run multiple commands via /bin/sh (no bashisms)] *************************
changed: [localhost]

TASK [Show output in a single block...] ****************************************
ok: [localhost] => {
    "o.stdout": "-rw-r--r--. 1 root root   9 Aug  7  2006 /etc/host.conf\n-rw-r--r--. 1 root root 172 Jun 29 09:31 /etc/hosts\n-rw-r--r--. 1 root root  11 Jun 29 09:31 /etc/hostname"
}

TASK [... or one per line] *****************************************************
ok: [localhost] => (item=-rw-r--r--. 1 root root   9 Aug  7  2006 /etc/host.conf) => {
    "item": "-rw-r--r--. 1 root root   9 Aug  7  2006 /etc/host.conf"
}
ok: [localhost] => (item=-rw-r--r--. 1 root root 172 Jun 29 09:31 /etc/hosts) => {
    "item": "-rw-r--r--. 1 root root 172 Jun 29 09:31 /etc/hosts"
}
ok: [localhost] => (item=-rw-r--r--. 

This approach has its limits.

### Exercise:

  - what happens if you processa long pipeline with  [shell-output-01.yml](/edit/notebooks/exercise-07/shell-output-01.yml) ? 
  
Exercise: modify [shell-output-01.yml](/edit/notebooks/exercise-07/shell-output-01.yml) so that:

  - every command stdout/stderr is redirected to a given file
  - before and after every command output print  a header and a footer (eg. the expected output is like
  
```
--- START COMMAND: cat /etc/resolv.conf --
nameserver 172.17.0.1
-- END COMMAND: cat /etc/resolv.conf
```

HINTS:

  - use shell redirection instead of `register`
  - use with_items to process many different commands 
  

In [18]:
!ansible-playbook shell-output-02.yml -i ../web


PLAY [localhost] ***************************************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [ensure /tmp/fetch is defined] ********************************************
changed: [localhost]

PLAY [web] *********************************************************************

TASK [Gathering Facts] *********************************************************
fatal: [ansible101_web_3]: UNREACHABLE! => {"changed": false, "msg": "Authentication or permission failure. In some cases, you may have been able to authenticate and did not have permissions on the remote directory. Consider changing the remote temp path in ansible.cfg to a path rooted in \"/tmp\". Failed command was: ( umask 77 && mkdir -p \"` echo ~/.ansible/tmp/ansible-tmp-1499011419.66-83364268236252 `\" && echo ansible-tmp-1499011419.66-83364268236252=\"` echo ~/.ansible/tmp/ansible-tmp-1499011419.66-83364268236252 `\" ), exited with result 1

## Templates

Creating files from templates.

include template files, enforcing policies

tagging

